In [4]:
import datetime
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import StaleElementReferenceException
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = Options()
# 최대화면으로 설정
options.add_argument('--start-maximized')
# 꺼짐 방지
options.add_experimental_option("detach", True)
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

#최대 30일까지 지원. 그 이후에는 과거 데이터로 예측, DB 구축해서 해결
now = datetime.date.today()

#현재 날짜부터 검색
now_year_date = now + datetime.timedelta(days=38)



#목표한 날짜. 30일정도 잡으면 될 것 같습니다
target_date = 30

#호텔명,입실,퇴실,지역,숙박유형,가격
place_list = []
area_dict = {
        '3ATokyo': '도쿄',
        '3AOsaka': '오사카',
        '3ASapporo': '삿포로',
        '3AOkinawa_Islands': '오키나와',
        '3AFukuoka': '후쿠오카'
    }
#호텔 / 게스트하우스,캡슐호텔,호스텔 / 리조트,펜션,료칸
ptype_dict = {
    '0' : '호텔',
    '16%2C28%2C5' : '게스트하우스,캡슐호텔,호스텔',
    '7%2C34%2C11' : '리조트,펜션,료칸'
}
for area in area_dict.keys():
    for ptype in ptype_dict.keys():
        for i in range(target_date):
            arr_date1 = now_year_date + datetime.timedelta(days=i)
            arr_date2 = arr_date1 + datetime.timedelta(days=1)
            url1 = f"https://hotels.naver.com/list?placeFileName=place%{area}&adultCnt=1&checkIn={arr_date1}&checkOut={arr_date2}&includeTax=false&sortField=popularityKR&sortDirection=descending&propertyTypes={ptype}"
            driver.get(url1)
            time.sleep(15)
            s = 0
            while s < 5:
                try:
                    data = driver.find_elements(By.CLASS_NAME, 'SearchList_HotelItem__aj2GM')

                    # 만약 어느 하나가 비어있으면 문제가 있는 것이므로 넘어갑니다
                    for element in data:
                        hname=element.find_element(By.CLASS_NAME,'Detail_title__40_dz').text
                        price = element.find_element(By.CLASS_NAME,'Price_show_price__iQpms').text
                        score = element.find_element(By.CLASS_NAME,'Detail_score__UxnqZ').text
                        star = element.find_element(By.CLASS_NAME,'Detail_grade__y5BmJ').text
                        price = re.sub(r"[^\d]", "", price)
                        place_list.append([hname,arr_date1,arr_date2,area_dict[area],ptype_dict[ptype],score,star,price])
                        print(hname,arr_date1,arr_date2,area_dict[area],ptype_dict[ptype],score,star,price)
    
                    driver.find_element(By.CLASS_NAME,'Pagination_next__OzkO7').click()
                    time.sleep(15)
                    s+=1

                except NoSuchElementException:
                    break


            print(len(place_list))

신주쿠 워싱턴 호텔 도쿄 메인 2023-09-01 2023-09-02 도쿄 호텔 7.4 3성급 89677
호텔 그레이스리 신주쿠 2023-09-01 2023-09-02 도쿄 호텔 8.6 4성급 142305
호텔 그레이스리 긴자 2023-09-01 2023-09-02 도쿄 호텔 8.2 4성급 97923
포 시즌스 호텔 도쿄 앳 오테마치 2023-09-01 2023-09-02 도쿄 호텔 9.0 5성급 976200
게이오 플라자 호텔 도쿄 2023-09-01 2023-09-02 도쿄 호텔 8.4 4성급 190251
온센 료칸 유엔 신주쿠 2023-09-01 2023-09-02 도쿄 호텔 8.6 3성급 107035
파크 하얏트 도쿄 2023-09-01 2023-09-02 도쿄 호텔 9.1 5성급 975850
더 아오야마 그랜드 호텔 2023-09-01 2023-09-02 도쿄 호텔 9.1 4성급 490474
호텔 알라만다 아오야마 도쿄 2023-09-01 2023-09-02 도쿄 호텔 8.5 5성급 256514
토부 호텔 레반트 도쿄 2023-09-01 2023-09-02 도쿄 호텔 8.8 4성급 115091
임페리얼 호텔 도쿄 2023-09-01 2023-09-02 도쿄 호텔 9.0 5성급 413517
JR-이스트 호텔 메츠 시부야 2023-09-01 2023-09-02 도쿄 호텔 8.4 3성급 192292
도쿄 프린스 호텔 2023-09-01 2023-09-02 도쿄 호텔 8.5 4성급 139354
쉐라톤 그랜드 도쿄 베이 호텔 2023-09-01 2023-09-02 도쿄 호텔 8.0 5성급 259354
하얏트 센트릭 긴자 도쿄 2023-09-01 2023-09-02 도쿄 호텔 8.8 4성급 552213
다이이치 호텔 도쿄 2023-09-01 2023-09-02 도쿄 호텔 8.8 4성급 201924
신주쿠 그란벨 호텔 2023-09-01 2023-09-02 도쿄 호텔 7.9 4성급 86977
렘 롯폰기 2023-09-01 2023-09-02 도쿄 호텔 8.6 3성

In [5]:
hotel_list=place_list
len(place_list)

5524

In [31]:
columns = ['호텔명', '입실', '퇴실', '지역', '숙박유형','별점','등급', '가격']
df = pd.DataFrame(hotel_list, columns=columns)
df

,호텔명,입실,퇴실,지역,숙박유형,별점,등급,가격
0,신주쿠 워싱턴 호텔 도쿄 메인,2023-09-01,2023-09-02,도쿄,호텔,7.4,3성급,89677
1,호텔 그레이스리 신주쿠,2023-09-01,2023-09-02,도쿄,호텔,8.6,4성급,142305
2,호텔 그레이스리 긴자,2023-09-01,2023-09-02,도쿄,호텔,8.2,4성급,97923
3,포 시즌스 호텔 도쿄 앳 오테마치,2023-09-01,2023-09-02,도쿄,호텔,9.0,5성급,976200
4,게이오 플라자 호텔 도쿄,2023-09-01,2023-09-02,도쿄,호텔,8.4,4성급,190251
...,...,...,...,...,...,...,...,...
5519,이마자토 료칸,2023-09-29,2023-09-30,오사카,"리조트,펜션,료칸",8.4,2성급,30503
5520,호텔 발리 언 리조트 신사이바시 (성인 전용),2023-09-29,2023-09-30,오사카,"리조트,펜션,료칸",8.5,2성급,164492
5521,야마토야 혼텐 료칸 오사카,2023-09-30,2023-10-01,오사카,"리조트,펜션,료칸",7.8,3성급,197790
5522,이마자토 료칸,2023-09-30,2023-10-01,오사카,"리조트,펜션,료칸",8.4,2성급,34162


In [16]:
# DataFrame을 CSV 파일로 저장
df.to_csv('hotel_list_new.csv', index=False, encoding='utf-8-sig')

In [34]:
# CSV 파일을 DataFrame으로 불러오기
df1 = pd.read_csv('hotel_list_new.csv')

# DataFrame 확인
df1

,호텔명,입실,퇴실,지역,숙박유형,별점,등급,가격
0,신주쿠 워싱턴 호텔 도쿄 메인,2023-09-01,2023-09-02,도쿄,호텔,7.4,3성급,89677
1,호텔 그레이스리 신주쿠,2023-09-01,2023-09-02,도쿄,호텔,8.6,4성급,142305
2,호텔 그레이스리 긴자,2023-09-01,2023-09-02,도쿄,호텔,8.2,4성급,97923
3,포 시즌스 호텔 도쿄 앳 오테마치,2023-09-01,2023-09-02,도쿄,호텔,9.0,5성급,976200
4,게이오 플라자 호텔 도쿄,2023-09-01,2023-09-02,도쿄,호텔,8.4,4성급,190251
...,...,...,...,...,...,...,...,...
5519,이마자토 료칸,2023-09-29,2023-09-30,오사카,"리조트,펜션,료칸",8.4,2성급,30503
5520,호텔 발리 언 리조트 신사이바시 (성인 전용),2023-09-29,2023-09-30,오사카,"리조트,펜션,료칸",8.5,2성급,164492
5521,야마토야 혼텐 료칸 오사카,2023-09-30,2023-10-01,오사카,"리조트,펜션,료칸",7.8,3성급,197790
5522,이마자토 료칸,2023-09-30,2023-10-01,오사카,"리조트,펜션,료칸",8.4,2성급,34162
